In [24]:
from unsloth import FastLanguageModel
import torch

SYSTEM_PROMPT = "Eres un modelo entrenado para generar resúmenes institucionales de actas parlamentarias. Los resúmenes deben estar redactados en lenguaje formal-administrativo, sin juicios de valor, y seguir una estructura clara."
INSTRUCTION = "Redacta un resumen institucional en español del siguiente documento. Mantén un lenguaje objetivo, enfocado en los hechos y acuerdos:"

model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name="meta-llama/Llama-3.2-1B-Instruct",
    model_name="BSC-LT/salamandra-2b-instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = False, # quantization
)
tokenizer.clean_up_tokenization_spaces = False

==((====))==  Unsloth 2025.5.9: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4070 SUPER. Num GPUs = 1. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load BSC-LT/salamandra-2b-instruct as a legacy tokenizer.


In [58]:
from transformers import TextStreamer

import re

def extract_clean_assistant_response(full_text: str) -> str:
    # Buscar el último bloque <|assistant|>
    assistant_start = full_text.rfind("assistant")
    if assistant_start == -1:
        assistant_content = full_text
    else:
        assistant_content = full_text[assistant_start + len("assistant"):]

    # Eliminar los bloques <think>...</think> si existen
    assistant_content = re.sub(r"<think>.*?</think>", "", assistant_content, flags=re.DOTALL)

    # Eliminar espacios extra al principio y al final
    return assistant_content.strip()

def apply_chat_template(sample, tokenizer):
    """
    Apply a chat template to the sample.
    """
    # Define the chat template
    empty_prompt = f"{INSTRUCTION}\n ##Documento {{document}}\n ##Resumen:"
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": empty_prompt.format(document=sample["document"])},
    ]
    
    # Format the chat template with the sample text
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


## generate a summary
def generate_summary_streamer(model, tokenizer, sample, max_new_tokens=100):
    FastLanguageModel.for_inference(model)
    if "inference_prompt" in sample:
        inputs = tokenizer(sample["inference_prompt"], return_tensors="pt").to(model.device)
    else:
        prompt = apply_chat_template(sample, tokenizer)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    text_streamer = TextStreamer(tokenizer)
    final_text = ""
    for token in model.generate(**inputs, streamer = text_streamer, max_new_tokens = max_new_tokens):
        print(token)
        pass

def generate_summary(model, tokenizer, sample, max_new_tokens=100):
    FastLanguageModel.for_inference(model)
    if "inference_prompt" in sample:
        inputs = tokenizer(sample["inference_prompt"], return_tensors="pt").to(model.device)
    else:
        prompt = apply_chat_template(sample, tokenizer)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=max_new_tokens, 
        temperature = 0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,  # Penaliza repetir palabras
        no_repeat_ngram_size=4,  # Evita repetir grupos de 4 palabras
        do_sample=True,          # Activar muestreo si no lo tienes ya por defecto

    )  # Adjust max_new_tokens as needed
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    summary = generated_text.split("##Resumen:")[-1].strip()
    if tokenizer.chat_template:
        summary = extract_clean_assistant_response(summary)
    return summary

In [26]:
# More info about parameters: https://huggingface.co/docs/peft/v0.11.0/en/package_reference/lora#peft.LoraConfig
target_modules =  ["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"]

# When adding special tokens
train_embeddings = False

if train_embeddings:
  target_modules = target_modules + ["lm_head"]

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # rank of lora matrices according to paper not much loss when set relatively low
    target_modules = target_modules,  # On which modules of the llm the lora weights are used
    lora_alpha = 2*16, # scales the weights of the adapters (more influence on base model), 16 was recommended on reddit
    lora_dropout = 0, # Default on 0.05 in tutorial but unsloth says 0 is better
    bias = "none",    # "none" is optimized
    use_gradient_checkpointing = "unsloth", #"unsloth" for very long context, decreases vram
    random_state = 3407,
    use_rslora = False,  # scales lora_alpha with 1/sqrt(r), huggingface says this works better
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.9 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [27]:
import pandas as pd
from datasets import Dataset
FOLDER = "sum/test_summary_normal.xlsx"

df = pd.read_excel(FOLDER, engine='openpyxl')
# convert to dataset
dataset = Dataset.from_pandas(df) 
print(dataset)

Dataset({
    features: ['document', 'expected_summary', 'generated_summary', 'language', 'time'],
    num_rows: 20
})


In [28]:
EOS_TOKEN = tokenizer.eos_token

empty_prompt = f"{INSTRUCTION}\n{{document}}\n\nResumen:"

def formatting_prompts_instruction(examples):
  training_prompts = []
  inference_prompts = []
  summaries = []
  for doc, sum in zip(examples["document"] , examples["expected_summary"]):
      inference_prompt = empty_prompt.format(document=doc)
      real_sum = sum.strip()
      training_prompt = inference_prompt + sum + EOS_TOKEN
      training_prompt = training_prompt.replace("\n", " ")  # Remove newlines for better tokenization
      training_prompt = training_prompt.strip()  # Remove leading/trailing spaces
      training_prompts.append(training_prompt)
      inference_prompts.append(inference_prompt)
      summaries.append(real_sum)

  return { "text" : training_prompts, 
           "inference_prompt" : inference_prompts,
           "expected_summary" : summaries }

In [29]:
if tokenizer.chat_template:
    print("Using chat template for formatting prompts")
    def formatting_func(example):
        empty_prompt = f"{INSTRUCTION}\n{{document}}\n"
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": empty_prompt.format(document=example["document"])},
            {"role": "assistant", "content": example["expected_summary"]}
        ]
        return tokenizer.apply_chat_template(messages, tokenize=False)
    dataset_train = dataset.map(lambda x: {"text": formatting_func(x)})
else:
    dataset_train = dataset.map(formatting_prompts_instruction, batched=True, remove_columns=dataset.column_names)
    print(dataset_train)

Using chat template for formatting prompts


Map: 100%|██████████| 20/20 [00:00<00:00, 3561.89 examples/s]


In [30]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # process 4 batches before updating parameters (parameter update == step)
        num_train_epochs = 2, # between 1 - 3 to prevent overfitting
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        warmup_ratio = 0.03, # 3% of the total steps
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    formatting_func=lambda x: x["text"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    args = args,
)

Unsloth: Tokenizing ["text"]: 100%|██████████| 20/20 [00:00<00:00, 102.78 examples/s]


In [31]:
def count_trainable_params(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    percentage = 100 * trainable_params / total_params
    return trainable_params, total_params, percentage

trainable_params, total_params, percentage = count_trainable_params(model)
print(f"Trainable parameters: {trainable_params:,} / Total parameters: {total_params:,} ({percentage:.2f}%)")

Trainable parameters: 14,917,632 / Total parameters: 2,268,407,808 (0.66%)


In [50]:
import torch

torch.cuda.reset_peak_memory_stats()

trainer_stats = trainer.train()
peak_memory = torch.cuda.max_memory_allocated() / (1024**3)  # en GB

print(f"Memoria máxima GPU usada: {peak_memory:.2f} GB")

## clean the memory of GPU
torch.cuda.empty_cache()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20 | Num Epochs = 2 | Total steps = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,917,632/2,268,407,808 (0.66% trained)


Step,Training Loss
1,2.097900
2,1.990400
3,1.688500
4,1.966100


Memoria máxima GPU usada: 4.81 GB


In [51]:
# Tiempo total en segundos
print("Tiempo total de entrenamiento:", trainer_stats.metrics["train_runtime"], "segundos")
print("Velocidad:", trainer_stats.metrics["train_samples_per_second"], "ejemplos/segundo")

Tiempo total de entrenamiento: 6.8621 segundos
Velocidad: 5.829 ejemplos/segundo


In [52]:
stats_of_trainer = {
    "peak_memory (GB)": peak_memory,
    "train_runtime": trainer_stats.metrics["train_runtime"],
    "train_samples_per_second": trainer_stats.metrics["train_samples_per_second"],
    "trainable_params": trainable_params,
}

print("Estadísticas del entrenamiento:", stats_of_trainer)

Estadísticas del entrenamiento: {'peak_memory (GB)': 4.806070804595947, 'train_runtime': 6.8621, 'train_samples_per_second': 5.829, 'trainable_params': 14917632}


In [53]:
sample = dataset_train[0]
print("Sample document:", sample)
excepted_summary = sample["expected_summary"]

Sample document: {'document': 'Esta sesión del parlamento se realizó el 14/05/2013. · 8L/PO/P-0916 Pregunta del señor diputado don Fabián Atamán Martín Martín, del Grupo Parlamentario Mixto, sobre los precios de las viviendas protegidas, dirigida a la señora consejera de Cultura, Deportes, Políticas Sociales y Vivienda. El señor presidente: Siguiente pregunta: del señor diputado don Fabián Atamán Martín, del Grupo Mixto, también dirigida a la señora consejera de Cultura, Deportes, Políticas Sociales y Vivienda. Don Fabián. El señor Martín Martín (Desde su escaño): Muchas gracias, señor presidente. Señora consejera, ¿qué valoración le merece que la evolución de los precios de las viviendas protegidas esté por encima de los de las viviendas libres? Muchas gracias. El señor presidente: Gracias, don Fabián. Señora consejera, doña Inés Rojas. La señora consejera de Cultura, Deportes, Políticas Sociales y Vivienda (Rojas de León) (Desde su escaño): Gracias, señor presidente. Señoría, decirle

In [56]:
text = generate_summary_streamer(model, tokenizer, sample, max_new_tokens=4080)

<s><|im_start|>system
Eres un modelo entrenado para generar resúmenes institucionales de actas parlamentarias. Los resúmenes deben estar redactados en lenguaje formal-administrativo, sin juicios de valor, y seguir una estructura clara.<|im_end|>
<|im_start|>user
Redacta un resumen institucional en español del siguiente documento. Mantén un lenguaje objetivo, enfocado en los hechos y acuerdos:
 ##Documento Esta sesión del parlamento se realizó el 14/05/2013. · 8L/PO/P-0916 Pregunta del señor diputado don Fabián Atamán Martín Martín, del Grupo Parlamentario Mixto, sobre los precios de las viviendas protegidas, dirigida a la señora consejera de Cultura, Deportes, Políticas Sociales y Vivienda. El señor presidente: Siguiente pregunta: del señor diputado don Fabián Atamán Martín, del Grupo Mixto, también dirigida a la señora consejera de Cultura, Deportes, Políticas Sociales y Vivienda. Don Fabián. El señor Martín Martín (Desde su escaño): Muchas gracias, señor presidente. Señora consejera,

Preguntas dirigidas a la señora consejera de Cultura, Deportes, Políticas Sociales y Vivienda acerca de la evolución de los precios de las viviendas protegidas y la importancia de reducir dichos precios. Respuesta destacando la reducción del precio de la vivienda protegida llevada a cabo por el Gobierno canario durante el último lustro. Además, se menciona la existencia de viviendas públicas disponibles en diferentes islas y la reducción de los precios públicos de dichas viviendas. Finalmente, se argumenta el aumento de la demanda por parte de jóvenes interesados en adquirir sus primeras viviendas y la necesidad de mantener el equilibrio entre el precio de la vivienda protegida y la vivienda libre. El informe concluye que el gobierno should have taken action to reduce the price of housing in the private market as well.<|im_end|>
tensor([     1,      4,   5027,  ...,  10032, 255655,      5], device='cuda:0')


In [57]:
print("text:", text)

text: None


In [59]:
text = generate_summary(model, tokenizer, sample, max_new_tokens=4080)
print("Generated summary:", text)

Generated summary: Los precios de las casas protegidas son superiores a los de las casoplones. Un 60 % de quienes viven en viviendas protegidas cobran menos de 1400 €, frente a solo el 24 % que gana más de 30.00 €. Mientras, el precio medio de las VPPs ha caído un 12 % desde 2010, aunque siguen estando algo por encima de las normales. Las diferencias salariales entre ambas opciones han ido disminuyendo. Solo el 3,4 % de los hogares canarios vive en casa propia. Los expertos insisten en que la construcción residencial debe ser accesible e incluir criterios sociales. La crisis ha provocado un aumento de demanda y nuevos modelos de acceso a la propiedad, pero sigue habiendo familias "por debajo". Además, los cambios normativos han abaratado costes, pero encarecido garantías. Ahora hay más hipotecas variables y préstamos personales para financiar la compra. Aunque el precio haya subido, sólo el 3'4% vive en su hogar propio. El Banco Santander concede más dinero que BBVA. Los bancos ofrecen